In [520]:
import pandas as pd 
import math
import numpy as np

In [497]:
plan=[
['P1',150,150,150,150],
['P2',100,100,100,100],
['P3',100,100,50,50]]
plan=pd.DataFrame(plan,columns=['Pr',1,  2,  3,  4  ])
plan.index=plan['Pr']
plan.drop(columns=['Pr'],inplace=True)

Periode=0

plan

,1,2,3,4
Pr,,,,
P1,150,150,150,150
P2,100,100,100,100
P3,100,100,50,50


In [527]:
bestand_df=pd.DataFrame(np.array(
[[21, 22, 23, 24, 25,   27, 28,   32, 33, 34,    35, 36, 37, 38, 39, 40, 41,  42, 43,   44, 45, 46, 47, 48,   52,  53,  57, 58,    59],
[150,200,200,3650,5800,1800,4500,2700,900,22000,3600,900,900,300,900,900,900,1800,1900,2700,900,900,900,1800,600,22000,600,22000,1800]]).T)

wartschlange=best_df=pd.DataFrame(np.array(
[[],
[]]).T)

merge_bestand=bestand_df.join(wartschlange,on=0,rsuffix="_")
merge_bestand

,0,1,0_,1_
0,21,1500,NaN,NaN
1,22,300,NaN,NaN
2,23,300,NaN,NaN
3,24,6100,NaN,NaN
4,25,3600,NaN,NaN
5,27,1800,NaN,NaN
6,28,4500,NaN,NaN
7,32,2700,NaN,NaN
8,33,900,NaN,NaN
9,34,22000,NaN,NaN


In [499]:
df=pd.read_csv('Data\\base_table_01.csv',delimiter=',',index_col='Unnamed: 0')
df['Bestand']=bestand
df['Lieferfrist']=df['Lieferfrist'].str.replace(',','.').astype('float')
df['Abweichung']=df['Abweichung'].str.replace(',','.').astype('float')

In [500]:
df['Bed_1']=df[['P1','P2','P3']]@plan[1]
df['Bed_2']=df[['P1','P2','P3']]@plan[2]
df['Bed_3']=df[['P1','P2','P3']]@plan[3]
df['Bed_4']=df[['P1','P2','P3']]@plan[4]
df['Menge']=df['Bestand']-(df['Bed_1']+df['Bed_2']+df['Bed_3']+df['Bed_4'])

In [501]:
def tt_0(x):
    bestand=x['Bestand']
    time_to0=5
    # print('beg ',bestand)
    # print('beg ',time_needed)

    for i in range(1,5):
        bestand-=x[f'Bed_{i}']
        #print(bestand,' bestand')
        if bestand<=0:
            time_to0=i-1
            #print(time_to0,' t0')
            break
    return time_to0

In [502]:
df['time_to0']=df.apply(tt_0,axis=1)

In [503]:
def order_na(x):
    time_needed=x['Lieferfrist']+x['Abweichung']
    #time_needed=math.ceil(time_needed)
    
    return 'schnell' if time_needed>=x['time_to0'] else 'normal'    

In [504]:
df['N/E']=df.apply(order_na,axis=1)
df['Lieferung Nötig']=df.apply(lambda x: 'Nein' if x['Menge']>=0 else 'Lieferung Nötig',axis=1)
df['Bestellmenge']=df.apply(lambda x : max([x['Diskontmenge'],(-x['Menge'])]),axis=1)
df['Bestellmenge']=df.apply(lambda x : x['Bestellmenge'] if x['Lieferung Nötig']=='Lieferung Nötig' else 0,axis=1)

In [506]:
df['Wareneingang']=df['Lieferfrist']+df['Abweichung']+Periode
df.to_csv('Data\\Bestellungen_01.csv',index=False)


#Einkaufsaufträge 
df[df['Lieferung Nötig']=='Lieferung Nötig'][['NR. K','Bestellmenge','N/E','Wareneingang']].to_csv(f'Bestellungen\\bestellungen_historie_{Periode}.csv',index=False)

In [510]:
df['lieferung_Empfehlung']=df['Wareneingang']>df['time_to0']

In [ ]:
df['Wareneingang']

In [ ]:
def funk(x):
    for i in range(1,5):
        bestand-=x[f'Bed_{i}']
        #print(bestand,' bestand')
        if bestand<=0:
            time_to0=i-1
            #print(time_to0,' t0')
            break

In [511]:
for i in range(1,5):
    df.apply(funk)
    df[f'plan_menge_{i}']

,NR. K,Lieferfrist,Abweichung,P1,P2,P3,Diskontmenge,Bestand,Bed_1,Bed_2,Bed_3,Bed_4,Menge,time_to0,N/E,Lieferung Nötig,Bestellmenge,Wareneingang,lieferung_Empfehlung
0,21,1.8,0.4,1,0,0,300,300,150,150,150,150,-300,1,schnell,Lieferung Nötig,300,2.2,True
1,22,1.7,0.4,0,1,0,300,300,100,100,100,100,-100,2,schnell,Lieferung Nötig,300,2.1,True
2,23,1.2,0.2,0,0,1,300,300,100,100,50,50,0,3,normal,Nein,0,1.4,False
3,24,3.2,0.3,7,7,7,6100,6100,2450,2450,2100,2100,-3000,2,schnell,Lieferung Nötig,6100,3.5,True
4,25,0.9,0.2,4,4,4,3600,3600,1400,1400,1200,1200,-1600,2,normal,Lieferung Nötig,3600,1.1,False
5,27,0.9,0.2,2,2,2,1800,1800,700,700,600,600,-800,2,normal,Lieferung Nötig,1800,1.1,False
6,28,1.7,0.4,4,5,6,4500,4500,1700,1700,1400,1400,-1700,2,schnell,Lieferung Nötig,4500,2.1,True
7,32,2.1,0.5,3,3,3,2700,2700,1050,1050,900,900,-1200,2,schnell,Lieferung Nötig,2700,2.6,True
8,33,1.9,0.5,0,0,2,900,900,200,200,100,100,300,5,normal,Nein,0,2.4,False
9,34,1.6,0.3,0,0,72,22000,22000,7200,7200,3600,3600,400,5,normal,Nein,0,1.9,False
